In [2]:
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

In [5]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# %%
mnist_train = datasets.FashionMNIST('./fashion_mnist/train', train=True,
                                    transform=transforms.Compose([transforms.ToTensor()]))
# %%
mnist_test = datasets.FashionMNIST('./fashion_mnist/test', train=False,
                                   transform=transforms.Compose([transforms.ToTensor()]))

mnist_train.data = mnist_train.data.float()
# %%
device = 'cpu'  # 'mps' if torch.has_mps else 'cpu'
# %%
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28 * 28, 2048),
    torch.nn.LeakyReLU(0.05),
    torch.nn.Linear(2048, 32),
    torch.nn.LeakyReLU(2),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(32, len(mnist_train.classes))
).to(device)


def test_accuracy(_model, _test_loader, _loss_func) -> [float, float]:
    """ Возвращает два числа -- точность и лосс """
    acc = 0
    _loss = 0
    with torch.no_grad():
        for j, (x, y) in enumerate(_test_loader, 1):
            x = x.to(device)
            y = y.to(device)
            pred = model(x)
            acc += torch.sum(torch.eq(pred.argmax(dim=1).long().to('cpu'), y.to('cpu')))
            _loss += _loss_func(pred, y)

    return acc / _test_loader.dataset.data.shape[0], _loss / j


# %%
def train(_model, train_data, test_data, epochs: int = 30, batch_size: int = 20_000) -> [torch.nn.Module, pd.DataFrame]:
    optimizer = torch.optim.Adam(_model.parameters(), weight_decay=0.00001)
    loss = torch.nn.CrossEntropyLoss()

    loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    log = pd.DataFrame(columns=['epoch', 'train_loss', 'test_accuracy', 'test_loss'])

    for i in range(epochs):
        epoch_loss = 0
        model.train()
        for j, (x, y) in enumerate(loader, 1):
            x = x.data.to(device)
            y = y.data.to(device)
            y_pred = _model(x.data)
            running_loss = loss(y_pred, y)
            running_loss.backward()

            epoch_loss += running_loss.item()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        test_acc, test_loss = test_accuracy(model, test_loader, loss)
        log.loc[i] = [i + 1, epoch_loss / j, test_acc.item(), test_loss.item()]

        print(f'EPOCH: {i + 1 :3d}  |  LOSS: {epoch_loss / j: .4f}', end='  |  ')
        print(f'TEST LOSS: {test_loss:0.4f}  |  TEST ACCURACY: {test_acc:0.4f}')

    return _model, log


# %%
model, log = train(model, mnist_train, mnist_test, 10)

RuntimeError: Dataset not found. You can use download=True to download it

## 1. Классификация предметов одежды (датасет Fashion MNIST)



### 1.1 Решить задачу классификации, не используя сверточные слои. 
* Предложить архитектуру модели для решения задачи
* Посчитать количество параметров модели.
* Обучить модель
* Вывести график функции потерь по эпохам. 
* Используя тестовое множество

  * Продемонстрировать работу модели: вывести несколько изображений, указать над ними правильный класс и класс, предсказанный моделью. 

  * Вывести матрицу ошибок.

  * Вывести значение accuracy на тестовом множестве.
* Сохранить модель

In [3]:
mnist_train = datasets.FashionMNIST('./fashion_mnist/train', train=True,
                                    transform=transforms.Compose([transforms.ToTensor()]))
mnist_train

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
mnist_test = datasets.FashionMNIST('./fashion_mnist/test', train=False,
                                   transform=transforms.Compose([transforms.ToTensor()]))
mnist_test

In [ ]:
device = 'cpu'  #  'mps' if torch.has_mps else 'cpu'

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28 * 28, 2048),
    torch.nn.LeakyReLU(0.05),
    torch.nn.Linear(2048, 32),
    torch.nn.LeakyReLU(2),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(32, len(mnist_train.classes))
).to(device)

number_params = 0
for param in model.parameters():
    number_params += torch.prod(torch.tensor(param.shape))

number_params  # Количество параметров в сети

In [ ]:
def test_accuracy(_model, _test_loader, _loss_func) -> [float, float]:
    """ Возвращает два числа -- точность и лосс """
    acc = 0
    _loss = 0
    n = 0
    with torch.no_grad():
        for j, (_x, _y) in enumerate(_test_loader, 1):
            _x = _x.to(device)
            _y = _y.to(device)
            pred = model(_x)
            acc += torch.sum(torch.eq(pred.argmax(dim=1).long(), _y))
            n += pred.shape[0]
            _loss += _loss_func(pred, _y)

    return acc / n, _loss / j

In [ ]:
def train(_model, train_data, test_data, epochs: int = 30, batch_size: int = 20_000) -> [torch.nn.Module, pd.DataFrame]:
    optimizer = torch.optim.Adam(_model.parameters(), weight_decay=0.0001)
    loss = torch.nn.CrossEntropyLoss()

    loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=8)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    _log = pd.DataFrame(columns=['epoch', 'train_loss', 'test_accuracy', 'test_loss'])

    for i in range(epochs + 1):
        epoch_loss = 0
        model.train()
        j = 1
        if i > 0:
            for j, (_x, _y) in enumerate(loader, 1):
                _x = _x.data.to(device)
                _y = _y.data.to(device)
                y_pred = _model(_x.data)
                running_loss = loss(y_pred, _y)
                running_loss.backward()

                epoch_loss += running_loss.item()

                optimizer.step()
                optimizer.zero_grad()

        model.eval()
        test_acc, test_loss = test_accuracy(model, test_loader, loss)
        _log.loc[i] = [i, epoch_loss / j, test_acc.item(), test_loss.item()]

        print(f'EPOCH: {i :3d}  |  LOSS: {epoch_loss / j: .4f}', end='  |  ')
        print(f'TEST LOSS: {test_loss:0.4f}  |  TEST ACCURACY: {test_acc:0.4f}')

    return _model, _log

In [ ]:
model, log = train(model, mnist_train, mnist_test, 3, 100)


In [ ]:
px.line(log, x='epoch', y=['train_loss', 'test_loss'], title='<b>LOSS PLOT</b>').show(renderer='png')


In [ ]:
px.line(log, x='epoch', y='test_accuracy', title='<b>TEST ACCURACY</b>').show(renderer='png')


In [ ]:
from sklearn.metrics import confusion_matrix

model.to('cpu')
right_ans = torch.zeros(mnist_test.data.shape[0]).to('cpu')
with torch.no_grad():
    i = 0
    for x, _ in DataLoader(mnist_test, batch_size=2000):
        x = x.to('cpu')
        right_ans[i:i + x.shape[0]] += model(x).argmax(dim=1)
        i += x.shape[0]

cm = confusion_matrix(mnist_test.targets.numpy(), right_ans.long().numpy())
px.imshow(cm).show(renderer='png')

In [ ]:
torch.save(model.state_dict(), 'models/mnist_linear')


In [ ]:
np.random.seed(21)
for i in np.random.randint(0, mnist_test.targets.shape[0], 3):
    id_class = mnist_test.targets[i]
    class_name = mnist_test.classes[id_class]

    id_predicted = model(mnist_test.data[i:i + 1].float()).argmax(dim=1).item()
    class_name_predicted = mnist_test.classes[id_predicted]
    print(f'CLASS : {id_class} ({class_name})  |  '
          f'PREDICTED CLASS : {id_predicted} ({class_name_predicted})')
    px.imshow(mnist_test.data[i], color_continuous_scale='gray').show(renderer='png')


### 1.2 Решить задачу 1.1, используя сверточную нейронную сеть. 
* Добиться значения accuracy на тестовом множестве не менее 90%
* Визуализировать результаты работы первого сверточного слоя

In [ ]:
mnist_train = datasets.FashionMNIST('./fashion_mnist/train', train=True,
                                    transform=transforms.Compose([transforms.ToTensor()]))
mnist_test = datasets.FashionMNIST('./fashion_mnist/test', train=False,
                                   transform=transforms.Compose([transforms.ToTensor()]))
mnist_train

In [ ]:
p = 0.2

model = torch.nn.Sequential(

    torch.nn.BatchNorm2d(1),

    torch.nn.Conv2d(1, 8, (3, 3), padding=1),
    torch.nn.BatchNorm2d(8),
    torch.nn.ReLU(),

    torch.nn.Conv2d(8, 64, (3, 3), padding=1),
    torch.nn.MaxPool2d((2, 2)),
    torch.nn.BatchNorm2d(64),
    torch.nn.ReLU(),

    torch.nn.Dropout(p),

    torch.nn.Flatten(),
    torch.nn.Linear(12544, 2048),
    torch.nn.Sigmoid(),
    torch.nn.Linear(2048, 10)

).to(device)

In [ ]:
number_params = 0
for param in model.parameters():
    number_params += torch.prod(torch.tensor(param.shape))

number_params  # Количество параметров в сети

In [ ]:
model, log = train(model, mnist_train, mnist_test, 3, 500)


In [ ]:
model.eval()


In [ ]:
from plotly.subplots import make_subplots

np.random.seed(21)

pic, target = next(iter(DataLoader(mnist_test, batch_size=1)))
fst_pic = model[:2](pic)[0]

print(f'CLASS : {target} ({mnist_test.classes[target]})')
for pic_i in fst_pic:
    fig = make_subplots(1, 2)
    px.imshow(pic_i.detach().numpy(), color_continuous_scale='gray').show(renderer='png')


##  2. Классификация изображений (датасет CIFAR 10) 


### 2.1 Решить задачу классификации, не используя сверточные слои. 

* Нормализовать данные (если необходимо)
* Предложить архитектуру модели для решения задачи
* Посчитать количество параметров модели.
* Обучить модель
* Вывести график функции потерь по эпохам. 
* Используя тестовое множество

  * Продемонстрировать работу модели: вывести несколько изображений, указать над ними правильный класс и класс, предсказанный моделью. 

  * Вывести матрицу ошибок.

  * Вывести значение accuracy на тестовом множестве.
* Сохранить модель

In [ ]:
class TensorReshape:
    def __call__(self, _x):
        return _x.reshape(3, 32, 32)

    def __repr__(self):
        return 'TensorReshape()'

In [ ]:
cifar_train = datasets.CIFAR10('./cifar_mnist/train', train=True, download=False,
                               transform=transforms.Compose([transforms.ToTensor(), TensorReshape()]))

cifar_test = datasets.CIFAR10('./cifar_mnist/train', train=False, download=False,
                              transform=transforms.Compose([transforms.ToTensor(), TensorReshape()]))
cifar_test

In [ ]:
model = torch.nn.Sequential(
    torch.nn.BatchNorm2d(3),
    torch.nn.Flatten(),
    torch.nn.Linear(3 * 32 * 32, 2048),
    torch.nn.LeakyReLU(0.05),
    torch.nn.Linear(2048, 32),
    torch.nn.LeakyReLU(2),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(32, len(cifar_train.classes))
).to(device)

number_params = 0
for param in model.parameters():
    number_params += torch.prod(torch.tensor(param.shape))

number_params  # Количество параметров в сети

In [ ]:
model, log = train(model, cifar_train, cifar_test, epochs=2, batch_size=500)


In [ ]:
import plotly.express as px

px.line(log, x='epoch', y=['train_loss', 'test_loss'], title='<b>LOSS PLOT</b>').show(renderer='png')


In [ ]:
px.line(log, x='epoch', y='test_accuracy', title='<b>TEST ACCURACY</b>').show(renderer='png')


In [ ]:
from sklearn.metrics import confusion_matrix

model.eval()
model.to('cpu')
right_ans = torch.zeros(cifar_test.data.shape[0]).to('cpu')
with torch.no_grad():
    i = 0
    for x, _ in DataLoader(cifar_test, batch_size=2000):
        x = x.to('cpu')
        right_ans[i:i + x.shape[0]] += model(x).argmax(dim=1)
        i += x.shape[0]

cm = confusion_matrix(cifar_test.targets, right_ans.long().numpy())
px.imshow(cm).show(renderer='png')

In [ ]:
torch.save(model.state_dict(), 'models/cifar_linear')


In [ ]:
model

In [ ]:
np.random.seed(21)
model.eval()
for i in np.random.randint(0, cifar_test.targets.__len__(), 3):
    id_class = cifar_test.targets[i]
    class_name = cifar_test.classes[id_class]
    image = torch.tensor(cifar_test.data[i:i + 1]).float().reshape(1, 3, 32, 32)

    id_predicted = model(image).argmax(axis=1).item()
    class_name_predicted = cifar_test.classes[id_predicted]
    print(f'CLASS : {id_class} ({class_name})  |  '
          f'PREDICTED CLASS : {id_predicted} ({class_name_predicted})')
    px.imshow(cifar_test.data[i], color_continuous_scale='gray').show(renderer='png')


### 2.2 Решить задачу 2.1, используя сверточную нейронную сеть. 
* Добиться значения accuracy на тестовом множестве не менее 70%.
* Визуализировать результаты работы первого сверточного слоя

In [ ]:
model = torch.nn.Sequential(

    torch.nn.BatchNorm2d(3),
    torch.nn.Conv2d(3, 16, (3, 3), padding=1),
    torch.nn.MaxPool2d((2, 2)),
    torch.nn.ReLU(),

    torch.nn.BatchNorm2d(16),
    torch.nn.Conv2d(16, 64, (3, 3), padding=1),
    torch.nn.MaxPool2d((2, 2)),
    torch.nn.ReLU(),

    torch.nn.BatchNorm2d(64),
    torch.nn.Conv2d(64, 64, (3, 3), padding=1),
    torch.nn.ReLU(),

    torch.nn.BatchNorm2d(64),
    torch.nn.Conv2d(64, 128, (3, 3), padding=1),
    torch.nn.MaxPool2d((2, 2)),
    torch.nn.ReLU(),

    torch.nn.Flatten(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(16 * 128, len(cifar_train.classes))
).to(device)

number_params = 0
for param in model.parameters():
    number_params += torch.prod(torch.tensor(param.shape))

number_params  # Количество параметров в сети

In [ ]:
model, log = train(model, cifar_train, cifar_test, epochs=3, batch_size=100)


In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=4)

images = model[:2](torch.tensor(cifar_test.data[20:21].reshape(1, 3, 32, 32)).float()).reshape(16, 32,
                                                                                               32).detach().numpy()

for n, image in enumerate(images):
    fig.add_trace(px.imshow(image).data[0], row=n // 4 + 1, col=n % 4 + 1)

layout = px.imshow(images[0], color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
# fig.update_xaxes(**layout.xaxis.to_plotly_json())
# fig.update_yaxes(**layout.yaxis.to_plotly_json())
fig.show(autoscale=False, height=1000, width=1000, renderer='png')


In [ ]:
px.imshow(cifar_test.data[20:21].reshape(32, 32, 3)).show(renderer='png')


## 3. Загрузка изображений из внешних источников

### 3.1 Решить задачу классификации обезьян (датасет [monkey.zip](https://disk.yandex.ru/d/OxYgY4S7aR6ulQ)).
* Загрузить архив с данными на диск
* Создать датасет на основе файлов при помощи `torchvision.datasets.ImageFolder`
* Преобразовать изображения к тензорами одного размера (например, 400х400). Потестировать другие преобразования из `torchvision.transforms`
* Предложить архитектуру модели для решения задачи. Обучить модель.
* Используя тестовое множество

  * Продемонстрировать работу модели: вывести несколько изображений, указать над ними правильный класс и класс, предсказанный моделью. 

  * Вывести матрицу ошибок.

  * Вывести значение accuracy на тестовом множестве.
  * Добиться значения accuracy на тестовом множестве не менее 60%

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
from tqdm import tqdm

zf = zipfile.ZipFile('drive/MyDrive/datasets/monkeys.zip')
for file in tqdm(zf.infolist()):
    zf.extract(file)

100%|██████████| 1371/1371 [00:16<00:00, 81.74it/s]


### 3.2 Решить задачу классификации собак и кошек (датасет [cats_dogs.zip](https://disk.yandex.ru/d/wQtt5O1JF9ctnA)).
* Загрузить архив с данными на диск
* Создать датасет на основе файлов при помощи `torchvision.datasets.ImageFolder`
* Преобразовать изображения к тензорами одного размера (например, 400х400). Потестировать другие преобразования из `torchvision.transforms`
* Предложить архитектуру модели для решения задачи. Обучить модель.
* Используя тестовое множество

  * Продемонстрировать работу модели: вывести несколько изображений, указать над ними правильный класс и класс, предсказанный моделью. 

  * Вывести матрицу ошибок.

  * Вывести значение accuracy на тестовом множестве.
  * Добиться значения accuracy на тестовом множестве не менее 80%

# 4. Transfer Learning

### 4.1 Решить задачу 3.1, воспользовавшись предобученной моделью VGG16
* Загрузить данные для обучения
* Преобразования: размер 224x224, нормализация с параметрами `mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)`
* Заменить последний полносвязный слой модели в соответствии с задачей
* Дообучить модель (не замораживать веса). Вычислить значение accuracy на тестовом множестве
* Дообучить модель (заморозить все веса, кроме последнего блока слоев (`classifier`)). 
* Вычислить значение accuracy на тестовом множестве.


### 4.2 Решить задачу 3.2, воспользовавшись подходящей предобученной моделью
* Не использовать VGG16 (вместо нее можно взять resnet18 или другую)
* Загрузить данные для обучения
* Преобразования: размер 224x224, нормализация с параметрами `mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)`
* Заменить последний полносвязный слой модели в соответствии с задачей
* Дообучить модель. 
* Вычислить значение accuracy на тестовом множестве (добиться значения не меньше 97-98%)